**Import Libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler

**Load the data**

In [2]:
data_file = 'letter-recognition.data'
columns = ['letter', 'x-box', 'y-box', 'width', 'height', 'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege', 'yegvx']

data = pd.read_csv(data_file, header=None, names=columns)

**Prepare the data**

In [3]:
label_encoder = LabelEncoder()
data['letter'] = label_encoder.fit_transform(data['letter'])

X = data.drop(columns=['letter']).values
y = data['letter'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**One-Hot Encoding**

In [4]:
def one_hot_encode(labels, num_classes):
    one_hot = np.zeros((labels.size, num_classes))
    one_hot[np.arange(labels.size), labels] = 1
    return one_hot

num_classes = len(np.unique(y))
y_train = one_hot_encode(y_train, num_classes)
y_test = one_hot_encode(y_test, num_classes)

**Xavier Initialization for weights**

In [5]:
def xavier_init(size):
    return np.random.randn(*size) * np.sqrt(1 / size[0])

**Neural Network with ReLu and Softmax**

In [6]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        self.learning_rate = learning_rate
        self.W1 = xavier_init((input_size, hidden_size))
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = xavier_init((hidden_size, output_size))
        self.b2 = np.zeros((1, output_size))
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def relu_derivative(self, z):
        return np.where(z > 0, 1, 0)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z))
        return exp_z / exp_z.sum(axis=1, keepdims=True)

    def forward(self, X):
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.softmax(self.Z2)
        return self.A2
    
    def compute_loss(self, y_true, y_pred):
        n_samples = y_true.shape[0]
        logp = - np.log(y_pred[range(n_samples), y_true.argmax(axis=1)])
        loss = np.sum(logp) / n_samples
        return loss

    def backpropagate(self, X, y_true, y_pred):
        n_samples = X.shape[0]
        
        dZ2 = y_pred - y_true
        dW2 = np.dot(self.A1.T, dZ2) / n_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * self.relu_derivative(self.A1)
        dW1 = np.dot(X.T, dZ1) / n_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples
        
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1

    def train(self, X, y, epochs=1000):
        losses = []
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.compute_loss(y, y_pred)
            losses.append(loss)
            self.backpropagate(X, y, y_pred)

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

        return losses
    
    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

**Train the Neural Network**

In [7]:
input_size = X_train.shape[1]
hidden_size = 128
output_size = num_classes
learning_rate = 0.1

nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
losses = nn.train(X_train, y_train, epochs=2000)

Epoch 0, Loss: 3.4632
Epoch 100, Loss: 1.5233
Epoch 200, Loss: 1.1866
Epoch 300, Loss: 1.0220
Epoch 400, Loss: 0.9174
Epoch 500, Loss: 0.8423
Epoch 600, Loss: 0.7844
Epoch 700, Loss: 0.7376
Epoch 800, Loss: 0.6985
Epoch 900, Loss: 0.6650
Epoch 1000, Loss: 0.6356
Epoch 1100, Loss: 0.6096
Epoch 1200, Loss: 0.5861
Epoch 1300, Loss: 0.5646
Epoch 1400, Loss: 0.5450
Epoch 1500, Loss: 0.5270
Epoch 1600, Loss: 0.5103
Epoch 1700, Loss: 0.4947
Epoch 1800, Loss: 0.4801
Epoch 1900, Loss: 0.4664


**OUTPUT - Prediction and Accuracy calculation**

In [9]:
sample_index = 69
class_labels = label_encoder.classes_
predicted_class = nn.predict(X_test[sample_index].reshape(1, -1))
predicted_letter = class_labels[predicted_class[0]]

print(f'Predicted class (numeric): {predicted_class[0]}')
print(f'Predicted letter: {predicted_letter}')

true_class = np.argmax(y_test[sample_index])
true_letter = class_labels[true_class]

print(f'True class (numeric): {true_class}')
print(f'True letter: {true_letter}')

def calculate_accuracy(X, y):
    correct_predictions = 0
    total_predictions = len(y)
    
    for i in range(total_predictions):
        predicted_class = nn.predict(X[i].reshape(1, -1))
        true_class = np.argmax(y[i])

        if predicted_class[0] == true_class:
            correct_predictions += 1

    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

accuracy = calculate_accuracy(X_test, y_test)
print(f'Accuracy on test set: {accuracy:.2f}%')

Predicted class (numeric): 17
Predicted letter: R
True class (numeric): 17
True letter: R
Accuracy on test set: 86.92%
